<a href="https://colab.research.google.com/github/kevkn2/projects-mine/blob/main/test%20capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unrar x "/content/TRAIN.rar" "/content/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/TRAIN.rar

Creating    /content/TRAIN                                            OK
Creating    /content/TRAIN/Kain Tenun Toraja                          OK
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja1.jpg                   0%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja10.jpg                  1%  2%  3%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja2.jpg                   6%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja3.jpg                  12%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja4.jpg                  13%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja5.jpg                  13% 14% 15%  OK 
Extracting  /content/TRAIN/Kain Tenun Toraja/toraja6.jpg                  16% 17% 18% 19% 20% 21% 22%  OK 
Extracting  /content/TRAI

In [ ]:
import tensorflow as tf
import numpy as np
import os
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        '/content/TRAIN',
        target_size=(300,300),
        batch_size=2,
        class_mode='categorical')

Found 30 images belonging to 3 classes.


In [ ]:
feature_extractor = hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/classification/1")
feature_extractor.trainable = True

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])
model.summary()
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 35, 35, 64)     

In [ ]:
img = image.load_img(r'C:\Users\FarhanNurHakim\Downloads\ulos109159496_82e693e9-d178-47ee-9cd4-d7c00488a497_664_664.jpg', target_size=(300,300))
x = image.img_to_array(img)
x = x/255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=2)
print(classes[0])

if classes[0]<0.5:
    print('ulos')
else:
    print('lurik')

tf.saved_model.save(model, r'D:\Experiment\TFExperiment\savedmodel')
converter = tf.lite.TFLiteConverter.from_saved_model('D:\Experiment\TFExperiment\savedmodel')
tflite_model = converter.convert()

tflite_model_file = pathlib.Path(r'D:\Experiment\TFExperiment\tmp\foo.tflite')
tflite_model_file.write_bytes(tflite_model)